In [1]:
#Multiple Agents using ConversableAgent and joining them as a list

import os
from dotenv import load_dotenv
from autogen import ConversableAgent

load_dotenv()
api_key = os.getenv("MY_KEY")

llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": api_key}]}

In [2]:
information_agent = ConversableAgent(
    name="Information Agent",
    system_message='''You are a helpful customer agent,
    you are here to get details when users start our product.
    Your job is to gather customer's name and the topic they want to generate questions on.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [3]:
question_agent = ConversableAgent(
    name="Question Agent",
    system_message='''You are a helpful agent,
    you are here to generate question based on our user's preferred topic.
    The question you generate must test the user's ability on that topic.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are an agent responsible for evaluating the
    answer given by the customer. If the answer is correct,
    appreciate the customer. If there is room for improvement
    provide feedback on how the answer can be improved.
    If it is completely wrong, provide the right answer.
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [5]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
chats = [
    {
        "sender": information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and the topic you want to generate questions on?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'topic': ''}",
        },
        "max_turns": 1,
        "clear_history" : True
    },
    {
        "sender": customer_proxy_agent,
        "recipient": question_agent,
        "message": 
                "Try answering the below question",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : True
    },    
    {
        "sender": question_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },        
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Feedback on your answer",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [7]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and the topic you want to generate questions on?

--------------------------------------------------------------------------------


c:\Users\gssabijp\AppData\Local\Programs\Python\Python311\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Information Agent):

Abinesh & Basic physics

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Question Agent):

Try answering the below question
Context: 
```json
{
  "name": "Abinesh",
  "topic": "Basic physics"
}
```

--------------------------------------------------------------------------------
Question Agent (to customer_proxy_agent):

What are the three laws of motion formulated by Sir Isaac Newton, and can you provide an example of each law in everyday life?

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

*********************************************************************